## Data Fetching Examples

### 1- Import the DataFetcher

In [8]:
from torchtrade.data_processors.ccxt_fetcher import CcxtFetcher
from torchtrade.envs.components.market import  Market
from torchtrade.envs.components.clock import Clock
from torchtrade.envs.components.trade import BoundedTrade
from datetime import datetime
import pandas as pd
import numpy as np

In [9]:
# Setting params
exchange = "binance"
symbols = ["BTC/USDT","ETH/USDT"]
timeFrame = "1d"

In [10]:
# Defining the market
datafetcher = CcxtFetcher(exchange,symbols,timeFrame)
df = datafetcher.fetch_data()


NetworkError: binance GET https://api.binance.com/api/v3/exchangeInfo

In [11]:
#Setting the clock
time_start = df.index[0][1]
time_end = df.index[-1][1]


clock = Clock(time_start,timeFrame)

mkt = Market(df)

#Market is observing the clock
clock.register_observer(mkt)


In [5]:
df

isTraded     open     high      low    close  \
symbol   timestamp                                                          
BTC/USDT 2017-08-17 04:00:00        1  4261.48  4280.56  4261.48  4261.48   
         2017-08-17 04:15:00        1  4261.48  4270.41  4261.32  4261.45   
         2017-08-17 04:30:00        1   4280.0  4310.07  4267.99  4310.07   
         2017-08-17 04:45:00        1  4310.07  4313.62  4291.37  4308.83   
         2017-08-17 05:00:00        1  4308.83  4328.69  4304.31  4304.31   
...                               ...      ...      ...      ...      ...   
ETH/USDT 2023-02-17 23:00:00        1  1698.73  1702.78  1694.06  1694.15   
         2023-02-17 23:15:00        1  1694.16  1696.99  1692.47  1694.72   
         2023-02-17 23:30:00        1  1694.72  1699.01  1694.04  1696.03   
         2023-02-17 23:45:00        1  1696.02  1698.53  1692.55  1693.87   
         2023-02-18 00:00:00        1  1693.81  1695.06  1692.14  1693.03   

                                 volume  
symbol   timestamp                       
BTC/USDT 2017-08-17 04:00:00   2.189061  
         2017-08-17 04:15:00   9.119865  
         2017-08-17 04:30:00  21.923552  
         2017-08-17 04:45:00  13.948531  
         2017-08-17 05:00:00   5.101153  
...                                 ...  
ETH/USDT 2023-02-17 23:00:00  4521.8699  
         2023-02-17 23:15:00  2509.9691  
         2023-02-17 23:30:00  2512.4672  
         2023-02-17 23:45:00  3878.4939  
         2023-02-18 00:00:00  6508.4308  

[386082 rows x 6 columns]

At each timestamp we create multiple trades with different combinations of TP and stop loss
at the same time our architecture enable previous trades update 

In [6]:
clock.reset()

stop_loss_levels = [1,2,3,4,5]
rr_levels = [1.5 , 2 , 2.5,3]

trades= []
while clock.timestamp < time_end : 
    data = mkt.data
    for index, row in data.iterrows():
        if row['isTraded'] == 1:
            trade = [[index[0],index[1],sl,rr] + [BoundedTrade(index[0],index[1],row['close'],sl,rr,mkt,clock.timeframe,0.9)] for sl in stop_loss_levels for rr in rr_levels] 
            trades+=trade
    clock.next()

trd = pd.DataFrame(trades, columns =['symbol', 'timestamp','stopLoss','riskReward','trade'])
trd.set_index(['symbol', 'timestamp', 'stopLoss', 'riskReward'], inplace=True)
trd["win/loss"] = trd['trade'].apply(lambda t : t.reward)
trd.to_csv('data.csv',index=False)


KeyboardInterrupt: 

NameError: name 'trd' is not defined